In [1]:
import json

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 15]

import numpy as np
np.set_printoptions(precision=3, linewidth=150)

from copy import copy
from itertools import combinations

import networkx as nx

from modules.classes import Road, Vertex, Graph

# Import Data

## Import original scale

In [2]:
with open("data/geodata/small_subset/roads_zh_curvy_9features.geojson", "r") as roadjson:
    original_roads = json.load(roadjson)

original_road_list = [Road(road) for road in original_roads['features']]

# for road in original_road_list:
#     print(road)

## Import generalized scale

In [3]:
with open("data/geodata/small_subset/roads_zh_curvy_smooth_9features.geojson", "r") as roadjson:
    smoothed_roads = json.load(roadjson)

smoothed_road_list = [Road(road) for road in smoothed_roads['features']]

# for road in smoothed_road_list:
#     print(road)

# Compute Translation Vectors and visualize

In [4]:
skip_vertices = 0
end = 1000000
visualize = False

exagg_x_axis = 1

if visualize:
    plt.figure()
    plt.axis('equal')

d = {}

# loop over roads indices
for i in range(len(original_road_list)):
    if i == i:
        d[i] = {
            'orig': {
                'properties': original_road_list[i].properties,
                'coordinates': []
            },
            'smooth': {
                'properties': smoothed_road_list[i].properties,
                'coordinates': []
            },
            'transl': {
                'coordinates': [],
                'vectors': []
            }}
        
        if visualize:
            original_road_list[i].exaggerate_axis(0, exagg_x_axis)
            smoothed_road_list[i].exaggerate_axis(0, exagg_x_axis)
            plt.plot(
                original_road_list[i].coordinates[skip_vertices:end, 0],
                original_road_list[i].coordinates[skip_vertices:end, 1],
                color='blue',
                zorder=0)
            plt.plot(
                smoothed_road_list[i].coordinates[skip_vertices:end, 0],
                smoothed_road_list[i].coordinates[skip_vertices:end, 1],
                color='red',
                zorder=0)

        # loop over road vertices' indices
        for j in range(original_road_list[i].properties['num_vertices']):

            if not (end > j >= skip_vertices):
                continue

            orig_vertex = np.array(original_road_list[i].coordinates[j])
            smooth_vertex = np.array(smoothed_road_list[i].coordinates[j])
            translation_vector = smooth_vertex - orig_vertex 

            d[i]['orig']['coordinates'].append(list(orig_vertex))
            d[i]['smooth']['coordinates'].append(list(smooth_vertex))
            d[i]['transl']['coordinates'].append([list(orig_vertex), [orig_vertex[0] + translation_vector[0], orig_vertex[1] + translation_vector[1]]])
            d[i]['transl']['vectors'].append(list(translation_vector))

            if visualize:
                plt.scatter(smooth_vertex[0], smooth_vertex[1], color=(248/256, 0, 5/256), s=10)
                plt.arrow(
                    orig_vertex[0], orig_vertex[1],
                    translation_vector[0], translation_vector[1],
                    color=(168/256, 0, 200/256),
                    head_width=6,
                    head_length=8,
                    length_includes_head=True)
                plt.scatter(orig_vertex[0], orig_vertex[1], color=(69/256, 109/256, 196/256), s=10)
            
            

if visualize:
    plt.savefig('transl', dpi=600)                      


# Save to GEOJSON

In [5]:
# transl_geojson_dict = {
# "type": "FeatureCollection",
# "name": "roads_zh_curvy_translation_vectors",
# "crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:EPSG::2056" } },
# "features": []
# }

# for i in range(len(d)):
#     transl_geojson_dict['features'].append(
#         { 
#             "type": "Feature",
#             "properties": {"id": i},
#             "geometry": {
#                 "type": "MultiLineString",
#                 "coordinates": d[i]['transl']} }
#     )


# with open("../data/geodata/small_subset/roads_zh_curvy_translation_vectors.geojson", "w") as transl_file:
#     json.dump(transl_geojson_dict, transl_file)

# Create Vertices

In [6]:
global_vertex_index = 0
orig_vertices = []

# iterate over roads
for i_road in range(len(d)):

    # create road-specific properties dictionary
    props = [
        d[i_road]['orig']['properties']['BEFAHRBARK'],
        d[i_road]['orig']['properties']['BELAGSART'],
        d[i_road]['orig']['properties']['OBJEKTART'],
        d[i_road]['orig']['properties']['length'],
        d[i_road]['orig']['properties']['num_vertices']]

    # iterate over vertices
    for i_vertex in range(d[i_road]['orig']['properties']['num_vertices']):

        if i_vertex == 0:
            connections = [global_vertex_index + 1]
        elif i_vertex == d[i_road]['orig']['properties']['num_vertices'] - 1:
            connections = [global_vertex_index - 1]
        else:
            connections = [global_vertex_index - 1, global_vertex_index + 1]
        

        orig_vertices.append(
            Vertex(
                index=              global_vertex_index,
                road_index=         i_road,
                coords=             d[i_road]['orig']['coordinates'][i_vertex],
                translation_vector= d[i_road]['transl']['vectors'][i_vertex],
                properties=         props,
                connections=        connections
            )
        )
        
        global_vertex_index += 1


index_change_dict = {}
num_vertices = len(orig_vertices)




# No spatial reduncancy

In [7]:
for i, j in combinations(iterable=range(num_vertices), r=2):
    if orig_vertices[i].coords == orig_vertices[j].coords:

        # if orig_vertices[i].index == 271:
        #     print("-----------------------------------")
        #     print(orig_vertices[i])
        #     print(orig_vertices[j])

        # combine all connections
        new_connections = []
        for connection in orig_vertices[i].connections + orig_vertices[j].connections:
            if connection not in new_connections:
                new_connections.append(connection)

        # dictionary: old -> new
        index_change_dict[orig_vertices[j].index] = orig_vertices[i].index

        # update connections
        orig_vertices[i].connections = copy(new_connections)
        # overwrite second vertex
        orig_vertices[j] = orig_vertices[i]


# create list with unique vertices
unique_vertices = list(dict.fromkeys(orig_vertices))

# change connections according to change-dictionary
for vertex in unique_vertices:
    for i_connection, connection in enumerate(vertex.connections):
        if connection in index_change_dict:
            vertex.connections[i_connection] = index_change_dict[connection]



# Ensure continuous indices

In [8]:
# create new fresh change dictionary
index_change_dict = {}

# ensure continuous indices
for i_vertex, vertex in enumerate(unique_vertices):
    if vertex.index != i_vertex:
        index_change_dict[vertex.index] = i_vertex
        vertex.index = i_vertex
    
    
# change connections according to change-dictionary
for vertex in unique_vertices:
    for i_connection, connection in enumerate(vertex.connections):
        if connection in index_change_dict:
            vertex.connections[i_connection] = index_change_dict[connection]
    

    
# index_change_dict


# Create Graph

In [9]:
g = Graph(unique_vertices)

G = nx.from_numpy_matrix(g.adj_matrix)
nx.set_node_attributes(G, g.node_dict)


Draw Graph

In [10]:

# plt.figure(figsize=(35, 35))
# nx.draw(G,  {n: G.nodes[n]['coords'] for n in G.nodes}, width=0.25, node_size=1, with_labels=True)
# plt.savefig('roads_networkx', dpi=300)


Save as Pickle

In [11]:
nx.write_gpickle(G, "data/pickles/nx_road_graph.pickle")